### This kernel uses output from [here](https://www.kaggle.com/jimpsull/explanation-of-the-current-training-set-ipynb), and multiple versions of [this kernel](https://www.kaggle.com/jimpsull/fast-test-set-reading-merged-with-fast-extractor).  There is also more background on the [SMOTE method here](https://www.kaggle.com/jimpsull/smote-training-set-test-feature-table).

In [ ]:
#!pip install modin
import numpy as np # linear algebra
#import modin.pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import os
print(os.listdir("../input"))
print(os.listdir("../input/explanation-of-the-current-training-set-ipynb"))
print(os.listdir("../input/all-the-data-to-build-the-testfeaturetable"))
print(os.listdir("../input/fastreadandextractl460-500"))
# Any results you write to the current directory are saved as output.

### I just listed directory contents for easy copy and paste

In [ ]:
trainDf=pd.read_csv("../input/explanation-of-the-current-training-set-ipynb/trainingSetToMatchCustomTestSet111518.csv")
trainDf.shape

### The shape is off - I'll deal with that in a minute.  I must have written index as a column

In [ ]:
testDfBegin=pd.read_csv("../input/all-the-data-to-build-the-testfeaturetable/Objects0Through3219999.csv")

### My extractor got a little tripped up on the last dataFrame, so I had to rerun it

In [ ]:
print(testDfBegin.shape)
testDfEnd=pd.read_csv("../input/fastreadandextractl460-500/testFeaturesFrom3220000TO3492890.csv")
print(testDfEnd.shape)

### Getting rid of index columns

In [ ]:
testDfBegin=testDfBegin.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
print(testDfBegin.shape)
testDfBegin.columns

### More of the same

In [ ]:
trainDf=trainDf.drop('Unnamed: 0', axis=1)
#print(testDfEnd.shape)

In [ ]:

testDfEnd=testDfEnd.drop(['Unnamed: 0'], axis=1)
print(testDfEnd.shape)

### Merge them for a complete test dataFrame

In [ ]:
testDf=testDfBegin.append(testDfEnd, sort=False)
testDf.shape

### Look to see if there are any nans that need to be dealt with

In [ ]:

#missing values in:
#distmod
#hostgal_specz - 
#deltaDetect - NaN would mean nothing was detected treat as zero
trainDf['deltaDetect'].fillna(0,inplace=True)
testDf['deltaDetect'].fillna(0, inplace=True)
testDf.describe()

## What do the missing distmod look like?

In [ ]:
nullFilter=testDf.loc[:,'distmod'].isnull()
missingDm=testDf.loc[nullFilter]
missingDm.head()
missingDm.describe()


## Same as the training data.  Missing distmod=intragalactic
### Get tigdf and tegdf (train intergalactic data frame, train extragalactic data frame)

In [ ]:
#tig is training intergalactic.  For test we'll just use ig
igdf=missingDm
egdf=testDf[nullFilter==False]

print(igdf.shape)
print(egdf.shape)

## We're going to want object_id out of the data when we train the model
- but we'll want it back when we put together our submission file for the 'real / test' set

In [ ]:
eobjdf=pd.DataFrame()
eobjdf.loc[:,'object_id']=egdf.loc[:,'object_id']
eobjdf.head()

iobjdf=pd.DataFrame()
iobjdf.loc[:,'object_id']=igdf.loc[:,'object_id']
iobjdf.head()

# We'll check again for NaNs
- we'll be eliminating hostgal_specz since its missing for most of the test data

In [ ]:
igdf.describe()

In [ ]:
egdf.describe()

### [Adapted from this kernel](https://www.kaggle.com/qianchao/smote-with-imbalance-data)**


In [ ]:
igdf.columns

### As I was adding markup I noticed a couple of oversights
- the transitory TF were supposed to be gone as they were rolled into outlierScore
- the object_id shouldn't be in the model
- I meant to replace the pbSectionValues with some aggregates on them

In [ ]:
#!pip install -U imbalanced-learn
#traindf=pd.read_csv('../input/trainingSetToMatchCustomTestSet111518.csv')
traindf=trainDf
print(traindf.shape)
print(traindf.head())

#hostgal_specz isn't in most rows of the test data
def dropUselessFeatures(df):
    print(df.shape)
    df=df.drop(['hephs','hepos','hepts','lephs','lepos','lepts','hostgal_specz',
                  'hmphs','hmpos','hmpts','lmphs','lmpos','lmpts',
                  'hlphs','hlpos','hlpts','llphs','llpos','llpts',
                'highEnergy_transitory_1.0_TF', 
                'highEnergy_transitory_1.5_TF',
                'lowEnergy_transitory_1.0_TF', 
                'lowEnergy_transitory_1.5_TF', 
               'object_id'], axis=1)
    
    #df.loc[:,'hMin']=np.min([df.loc[:,'heavg'], df.loc[:,'hmavg'],df.loc[:,'hlavg']])
    #df.loc[:,'hMax']=np.max([df.loc[:,'heavg'], df.loc[:,'hmavg'],df.loc[:,'hlavg']])
    #df.loc[:,'hVar']=np.average([df.loc[:,'hestd'], df.loc[:,'hmstd'],df.loc[:,'hlstd']])
    #df.loc[:,'hSpread']=(df.loc[:,'hMax']-df.loc[:,'hMin']) / df.loc[:,'hVar']
    
    #df.loc[:,'lMin']=np.min([df.loc[:,'leavg'], df.loc[:,'lmavg'],df.loc[:,'llavg']])
    #df.loc[:,'lMax']=np.max([df.loc[:,'leavg'], df.loc[:,'lmavg'],df.loc[:,'llavg']])
    #df.loc[:,'lVar']=np.average([df.loc[:,'lestd'], df.loc[:,'lmstd'],df.loc[:,'llstd']])
    #df.loc[:,'lSpread']=(df.loc[:,'lMax']-df.loc[:,'lMin']) / df.loc[:,'lVar']
    
    #df=df.drop(['hMin','hMax','hVar','lMin','lMax','lVar'],axis=1)
    
    print(df.shape)
    return df

traindf=dropUselessFeatures(traindf)



In [ ]:
traindf.head()
traindf.columns

In [ ]:
traindf.loc[:,'target']=traindf.loc[:,'target'].astype(str)

#from stacy's code
# move target to end
traindf = traindf[[c for c in traindf if c not in ['target']] + ['target']]
traindf.head()

#df[1].fillna(0, inplace=True)
traindf['deltaDetect'].fillna(0,inplace=True)
tigdf=traindf[traindf['hostgal_photoz']==0]
tegdf=traindf[traindf['hostgal_photoz']!=0]

print(tigdf.shape)
print(tegdf.shape)


tigdf=tigdf.drop(['hostgal_photoz', 'distmod', 'hostgal_photoz_err'], axis=1)
print(tigdf.shape)

## Let's pare down the test dataFrames as well

In [ ]:
igdf=dropUselessFeatures(igdf)
print(igdf.shape)
egdf=dropUselessFeatures(egdf)
print(egdf.shape)

In [ ]:
igdf=igdf.drop(['hostgal_photoz', 'distmod', 'hostgal_photoz_err'], axis=1)
print(igdf.shape)

## How badly is the data imbalanced?  Intergalactic:

In [ ]:
print('inter-galactic')
for theClass in tigdf.loc[:,'target'].unique():
    print('class ' + str(theClass) + ':')
    trueFilter=tigdf['target']==theClass
    print(trueFilter.sum())

## Extra-galactic

In [ ]:
print('extra-galactic')
for theClass in tegdf.loc[:,'target'].unique():
    print('class ' + str(theClass) + ':')
    trueFilter=tegdf['target']==theClass
    print(trueFilter.sum())

### I'm using [qianchao's](https://www.kaggle.com/qianchao) kernel as a starting point

In [ ]:
#https://www.kaggle.com/qianchao/smote-with-imbalance-data
#from sklearn.preprocessing import StandardScaler
Xig = np.array(tigdf.iloc[:, tigdf.columns != 'target'])
yig = np.array(tigdf.iloc[:, tigdf.columns == 'target'])
print('Shape of X: {}'.format(Xig.shape))
print('Shape of y: {}'.format(yig.shape))

## Repeat for Extra galactic

In [ ]:

Xeg = np.array(tegdf.iloc[:, tegdf.columns != 'target'])
yeg = np.array(tegdf.iloc[:, tegdf.columns == 'target'])
print('Shape of X: {}'.format(Xeg.shape))
print('Shape of y: {}'.format(yeg.shape))

## The first time I did it manually
- I copied [qianchao's](https://www.kaggle.com/qianchao) [kernel](https://www.kaggle.com/qianchao/smote-with-imbalance-data) and then modified the classes

In [ ]:
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

Xig_train, Xig_test, yig_train, yig_test = train_test_split(Xig, yig, test_size=0.3, random_state=0)

print("Number transactions X_train dataset: ", Xig_train.shape)
print("Number transactions y_train dataset: ", yig_train.shape)
print("Number transactions X_test dataset: ", Xig_test.shape)
print("Number transactions y_test dataset: ", yig_test.shape)

## Make sure you have no NaNs at this point

In [ ]:
print("Before OverSampling, counts of label '92': {}".format(sum(yig_train=='92')))
print("Before OverSampling, counts of label '65': {} \n".format(sum(yig_train=='65')))
print("Before OverSampling, counts of label '16': {}".format(sum(yig_train=='16')))
print("Before OverSampling, counts of label '6': {} \n".format(sum(yig_train=='6')))
print("Before OverSampling, counts of label '53': {}".format(sum(yig_train=='53')))

sm = SMOTE(random_state=2)
Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(Xig_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(yig_train_res.shape))

print("After OverSampling, counts of label '92': {}".format(sum(yig_train_res=='92')))
print("After OverSampling, counts of label '65': {}".format(sum(yig_train_res=='65')))
print("After OverSampling, counts of label '16': {}".format(sum(yig_train_res=='16')))
print("After OverSampling, counts of label '6': {}".format(sum(yig_train_res=='6')))
print("After OverSampling, counts of label '53': {}".format(sum(yig_train_res=='53')))


## We really need to make this a method
- We're going to have to do it over and over again because SMOTE should be done AFTER cross validation splitting
- and there's no guarantee every class will be in every cross validated sample

In [ ]:
def smoteAdataset(Xig, yig, test_size=0.2, random_state=0):
    
    Xig_train, Xig_test, yig_train, yig_test = train_test_split(Xig, yig, test_size=test_size, random_state=random_state)
    print("Number transactions X_train dataset: ", Xig_train.shape)
    print("Number transactions y_train dataset: ", yig_train.shape)
    print("Number transactions X_test dataset: ", Xig_test.shape)
    print("Number transactions y_test dataset: ", yig_test.shape)

    classes=[]
    for i in np.unique(yig):
        classes.append(i)
        print("Before OverSampling, counts of label " + str(i) + ": {}".format(sum(yig_train==i)))
        
    sm=SMOTE(random_state=2)
    Xig_train_res, yig_train_res = sm.fit_sample(Xig_train, yig_train.ravel())

    print('After OverSampling, the shape of train_X: {}'.format(Xig_train_res.shape))
    print('After OverSampling, the shape of train_y: {} \n'.format(yig_train_res.shape))
    
    for eachClass in classes:
        print("After OverSampling, counts of label " + str(eachClass) + ": {}".format(sum(yig_train_res==eachClass)))
        
    return Xig_train_res, yig_train_res, Xig_test, yig_test

Xeg_train_res, yeg_train_res, Xeg_test, yeg_test=smoteAdataset(Xeg, yeg)


## We're ready to start training models
- We have oversampling training sets for both inter-galactic and extra-galactic objects

# Let's recount what we have
- igdf and egdf are the 'real' dataFrames (we won't call them test anymore) for inter and extra galactic
- Xeg_train_res and yeg_train_res are the SMOTE training sets for extragalactic
- Xeg_test and yeg_test are the test split of the training data for extragalactic
- Xig_train_res and yig_train_res are the SMOTE training sets for intragalactic
- Xig_test and yig_test are the test split of the training data for intragalactic


In [ ]:
#from HW5
from sklearn.ensemble import GradientBoostingClassifier

def getGbm(X_encoded_train, Y1, X_encoded_test, Y2,
           nTrees=100, max_depth=5, min_node_size=5, verbose=0, learning_rate=0.05):

    gbm_clf = GradientBoostingClassifier(n_estimators=nTrees, loss='deviance', learning_rate=learning_rate, max_depth=max_depth, \
                                        min_samples_leaf=min_node_size)
    gbm_clf.fit(X_encoded_train, Y1)
    
    
    Y_test_hat = gbm_clf.predict_proba(X_encoded_test)
    #Accuracy = [1 for i in range(len(Y_test_hat)) if Y2.iloc[i] == Y_test_hat[i]]
    #Accuracy = round(float(np.sum(Accuracy))/len(Y_test_hat)*100,2)
    #rocAuc=roc_auc_score(Y2Vals, Y_test_hat)
    
    #Y1Vals=np.array(Y1)
    
    #Y_train_hat = clf.predict(X_encoded_train)
    #trainAcc = [1 for i in range(len(Y_train_hat)) if Y1.iloc[i] == Y_train_hat[i]]
    #trainAcc = round(float(np.sum(trainAcc))/len(Y_train_hat)*100,2)
    #trainAuc=roc_auc_score(Y1Vals, Y_train_hat)
    
    
    return gbm_clf, Y_test_hat
tiggbm_clf, tigY_test_hat=getGbm(Xig_train_res, yig_train_res, Xig_test, yig_test,
                                 nTrees=100, max_depth=5, min_node_size=5, verbose=0, learning_rate=0.05)

print(tiggbm_clf.feature_importances_)
#print("Accuracy on Testing Data = %.2f%%"%Accuracy)
#print("AUC for ROC curve on Testing Data = %.2f"%rocAuc)
#print("Accuracy on Training Data = %.2f%%"%trainAcc)
#print("AUC for ROC curve on Training Data = %.2f"%trainAuc)

### The method returned the actual model (which we'll use on the 'real / test' data
- it also returned predicted probabilities from which we could compute the loss function

In [ ]:
print(tigY_test_hat)
print(tiggbm_clf.classes_)

### Repeat for the extra-galactic data and model

In [ ]:
teggbm_clf, tegY_test_hat=getGbm(Xeg_train_res, yeg_train_res, Xeg_test, yeg_test,
                                 nTrees=100, max_depth=5, min_node_size=5, verbose=0, learning_rate=0.05)


In [ ]:
print(teggbm_clf.feature_importances_)

In [ ]:
print(tegY_test_hat)
print(teggbm_clf.classes_)

### Get the predictions based on the feature values for intra and extra galactic

In [ ]:
actualIgX=igdf.values
actualIgPredictions = tiggbm_clf.predict_proba(actualIgX)
#igPredictions = tiggbm_clf.predict_proba(X_encoded_test)

In [ ]:
actualEgX=egdf.values
actualEgPredictions = teggbm_clf.predict_proba(actualEgX)
#igPredictions = tiggbm_clf.predict_proba(X_encoded_test)

In [ ]:
actualIgPredictions.shape

In [ ]:
actualEgPredictions.shape

### We miss the object_id - I wanted it out of the model but I need it for the submission file
- we'll need to index them to merge them with the results arrays

In [ ]:
iobjdf.index=range(iobjdf.shape[0])
iobjdf.head()

eobjdf.index=range(eobjdf.shape[0])
eobjdf.head()

### We have to get everything into the submission file format

In [ ]:
igResultDf=pd.DataFrame(data=actualIgPredictions, columns=tiggbm_clf.classes_)
#igResultDf.loc[:,'object_id']=igdf.loc[:,'object_id']
igResultDf.loc[:,'object_id']=iobjdf.loc[:,'object_id']
igResultDf.head()

In [ ]:
egResultDf=pd.DataFrame(data=actualEgPredictions, columns=teggbm_clf.classes_)
#igResultDf.loc[:,'object_id']=igdf.loc[:,'object_id']
egResultDf.loc[:,'object_id']=eobjdf.loc[:,'object_id']
egResultDf.head()

## Merge them into one results set

In [ ]:
fulldf=igResultDf.append(egResultDf, sort=False)
print(fulldf.shape)

### To use the max_value I had to get object_id out
- in hindsight I should've waited to put it back in

In [ ]:
import copy
nobjdf=copy.deepcopy(fulldf)
nobjdf=nobjdf.drop('object_id', axis=1)
nobjdf['max_value'] = nobjdf.max(axis=1)
print(nobjdf.head())
#I chose 7000 arbitrarily because its about 1/5 of 1%

#arbGuess99=7000
#arb99Proba=1.00
#mysteryThresh=np.max(nobjdf.nsmallest(arbGuess99, 'max_value').loc[:,'max_value'])
#print(mysteryThresh)
#nobjdf['99']=0
#nobjdf.loc[nobjdf['max_value']<]


## Would've been more efficient to do it other ways but I'm just commenting
- and trying to change what I did after the fact

In [ ]:
averages=[]
for columns in nobjdf.columns:
    nobjdf[columns].fillna(0, inplace=True)
    #print(columns)
    averages.append(np.average(nobjdf.loc[:,columns]))
    #print(np.average(nobjdf.loc[:,columns]))
    
nobjdf.loc[:,'g99']=1-nobjdf.loc[:,'max_value']**2
#print(np.min(averages))
#print(nobjdf.head())
normalizingConstant = np.average(nobjdf.loc[:,'g99'])/np.min(averages)
print(normalizingConstant)

## This is arbitrary.  I'm saying class 99 is about as common as the most rare of the known classes

In [ ]:
fulldf.loc[:,'99']=nobjdf.loc[:,'g99']/normalizingConstant
fulldf.head()

In [ ]:
for cindex in fulldf.columns:
    fulldf[cindex].fillna(0, inplace=True)
    
fulldf.describe()

In [ ]:
def submitOrder(fulldf):
    
    deleteCols=[]
    for cindex in fulldf.columns:
        fulldf = fulldf.rename(columns={cindex: 'was'+str(cindex)})
        deleteCols.append('was'+str(cindex))
    
    newNames=[6,15, 16, 42, 52,53,62,
             64,65,67,88,90,92,95,99]
    
    #string column names seemed to cause a problem
    fulldf.loc[:,'object_id']=fulldf.loc[:,'wasobject_id']
    for name in newNames:
        fulldf.loc[:,'class_'+str(name)]=fulldf.loc[:,'was'+str(name)]
    
    fulldf=fulldf.drop(deleteCols, axis=1)
    return fulldf


submitdf=submitOrder(fulldf)
#for cindex in submitdf.columns:
#    if cindex != 'object_id':
#        submitdf.loc[:,cindex]=submitdf.loc[:,cindex].astype(float)
print(submitdf.shape)
print(submitdf.columns)
submitdf.head()

### We can submit to the competition directly from the output of this kernel

In [ ]:
submitdf.to_csv('fastFeatsOnly111718.csv', index=False)